Задание: https://docs.google.com/document/d/18bJyle9I2GCnxb4sygRHswLA_k6lLlLdW_Gatd_GRwM  
Доп. материалы:  
https://incompleteideas.net/book/bookdraft2017nov5.pdf  
https://github.com/udacity/deep-reinforcement-learning/blob/master/cheatsheet/cheatsheet.pdf  



# Advanced DL and RL: Домашнее задание 2  
Второе домашнее задание опять посвящено игре; его базовая часть, надеюсь, не слишком большая, но я добавил опциональную часть, которая, думаю, должна быть достаточно интересной для любого слушателя. Как обычно, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Любые комментарии, новые идеи и рассуждения на тему, как всегда, категорически приветствуются.  

## Часть первая: крестики-нолики при помощи Q-обучения  
В коде, прилагающемся к последней лекции про обучение с подкреплением, реализован Environment для крестиков-ноликов, в котором можно при инициализации указывать разные размеры доски и условия победы, а также функции для рисования, в том числе с указанием оценки различных действий. С этим окружением все задания и связаны.  

In [ ]:
import gym

import sys
from time import sleep
import tqdm.notebook as tq
import copy
from collections import defaultdict
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math
import random
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.integrate as integrate
from scipy.stats import multivariate_normal
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import statsmodels.api as sm
from matplotlib.colors import LogNorm
import pickle

from joblib import Parallel, delayed
import multiprocessing
from collections import namedtuple
from itertools import count

import cProfile
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('figure', **{'dpi': 300})

In [ ]:
N_ROWS, N_COLS, N_WIN = 3, 3, 3

Я допилил класс TicTacToe под себя. Добавил дополнительные методы. 

In [ ]:
class TicTacToe(gym.Env):
    def __init__(self, n_rows=N_ROWS, n_cols=N_COLS, n_win=N_WIN):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.n_win = n_win

        self.board = np.zeros((self.n_rows, self.n_cols), dtype=int)
        self.gameOver = False
        self.boardHash = None
        # ход первого игрока
        self.curTurn = 1
        self.emptySpaces = None
        
        self.reset()
        
    def getNumStates(self):
        return self.n_rows * self.n_cols * 3
    
    def getTotalNumActions(self):
        return self.n_rows * self.n_cols

    def getEmptySpaces(self):
        if self.emptySpaces is None:
            res = np.where(self.board == 0)
            self.emptySpaces = np.array([ (i, j) for i,j in zip(res[0], res[1]) ])
        return self.emptySpaces

    def makeMove(self, player, i, j):
        self.board[i, j] = player
        self.emptySpaces = None
        self.boardHash = None

    def getHash(self):
        if self.boardHash is None:
            self.boardHash = ''.join(['%s' % (x+1) for x in self.board.reshape(self.n_rows * self.n_cols)])
        return self.boardHash

    def isTerminal(self):
        # проверим, не закончилась ли игра
        cur_marks, cur_p = np.where(self.board == self.curTurn), self.curTurn
        for i,j in zip(cur_marks[0], cur_marks[1]):
            # print((i,j))
            win = False
            if i <= self.n_rows - self.n_win:
                if np.all(self.board[i:i+self.n_win, j] == cur_p):
                    win = True
            if not win:
                if j <= self.n_cols - self.n_win:
                    if np.all(self.board[i,j:j+self.n_win] == cur_p):
                        win = True
            if not win:
                if i <= self.n_rows - self.n_win and j <= self.n_cols - self.n_win:
                    if np.all(np.array([ self.board[i+k,j+k] == cur_p for k in range(self.n_win) ])):
                        win = True
            if not win:
                if i <= self.n_rows - self.n_win and j >= self.n_win-1:
                    if np.all(np.array([ self.board[i+k,j-k] == cur_p for k in range(self.n_win) ])):
                        win = True
            if win:
                self.gameOver = True
                return self.curTurn

        if len(self.getEmptySpaces()) == 0:
            self.gameOver = True
            return 0

        self.gameOver = False
        return None

    def printBoard(self):
        for i in range(0, self.n_rows):
            print('----'*(self.n_cols)+'-')
            out = '| '
            for j in range(0, self.n_cols):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('----'*(self.n_cols)+'-')

    def getState(self):
        return (self.getHash(), self.getEmptySpaces(), self.curTurn)

    def action_from_int(self, action_int):
        return [int(action_int // self.n_cols), int(action_int % self.n_cols)]

    def int_from_action(self, action):
        return action[0] * self.n_cols + action[1]
    
    def step(self, action):
        if self.board[action[0], action[1]] != 0:
            return self.getState(), -10, True, {}
        self.makeMove(self.curTurn, action[0], action[1])
        reward = self.isTerminal()
        self.curTurn = -self.curTurn
        return self.getState(), 0 if reward is None else reward, reward is not None, {}

    def reset(self):
        self.board = np.zeros((self.n_rows, self.n_cols), dtype=int)
        self.boardHash = None
        self.gameOver = False
        self.emptySpaces = None
        self.curTurn = 1
        return self.getState()
    
    def random_available_action(self):
        return random.choice(self.getEmptySpaces())
    
    def random_available_action_no(self):
        return self.int_from_action(self.random_available_action())
    
    def is_available_action(self, action):
        # print(action, self.getEmptySpaces().tolist())
        return action in self.getEmptySpaces().tolist()
    
    def is_available_action_int(self, action_no):
        return self.is_available_action(self.action_from_int(action_no))

In [ ]:
def plot_board(env, pi, showtext=True, verbose=True, fontq=20, fontx=60):
    '''Рисуем доску с оценками из стратегии pi'''
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    X, Y = np.meshgrid(np.arange(0, env.n_rows), np.arange(0, env.n_rows))
    Z = np.zeros((env.n_rows, env.n_cols)) + .01
    s, actions = env.getHash(), env.getEmptySpaces()
    if pi is not None and s in pi.Q:
        for i, a in enumerate(actions):
            Z[a[0], a[1]] = pi.Q[s][i]
    ax.set_xticks([])
    ax.set_yticks([])
    surf = ax.imshow(Z, cmap=plt.get_cmap('Accent', 10), vmin=-1, vmax=1)
    if showtext:
        for i,a in enumerate(actions):
            if pi is not None and s in pi.Q:
                ax.text( a[1] , a[0] , "%.3f" % pi.Q[s][i], fontsize=fontq, horizontalalignment='center', verticalalignment='center', color="w" )
#             else:
#                 ax.text( a[1] , a[0] , "???", fontsize=fontq, horizontalalignment='center', verticalalignment='center', color="w" )
    for i in range(env.n_rows):
        for j in range(env.n_cols):
            if env.board[i, j] == -1:
                ax.text(j, i, "O", fontsize=fontx, horizontalalignment='center', verticalalignment='center', color="w" )
            if env.board[i, j] == 1:
                ax.text(j, i, "X", fontsize=fontx, horizontalalignment='center', verticalalignment='center', color="w" )
    cbar = plt.colorbar(surf, ticks=[0, 1])
    ax.grid(False)
    plt.show()

def get_and_print_move(env, pi, s, actions, random=False, verbose=True, fontq=20, fontx=60):
    '''Делаем ход, рисуем доску'''
    plot_board(env, pi, fontq=fontq, fontx=fontx)
    if verbose and (pi is not None):
        if s in pi.Q:
            for i,a in enumerate(actions):
                print(i, a, pi.Q[s][i])
        else:
            print("Стратегия не знает, что делать...")
    if random:
        return np.random.randint(len(actions))
    else:
        return pi.getActionGreedy(s, len(actions))

In [ ]:
def plot_test_game(env, pi1, pi2, random_crosses=False, random_naughts=True, verbose=True, fontq=20, fontx=60):
    '''Играем тестовую партию между стратегиями или со случайными ходами, рисуем ход игры'''
    done = False
    env.reset()
    while not done:
        s, actions = env.getHash(), env.getEmptySpaces()
        if env.curTurn == 1:
            a = get_and_print_move(env, pi1, s, actions, random=random_crosses, verbose=verbose, fontq=fontq, fontx=fontx)
        else:
            a = get_and_print_move(env, pi2, s, actions, random=random_naughts, verbose=verbose, fontq=fontq, fontx=fontx)
        observation, reward, done, info = env.step(actions[a])
        if reward == 1:
            print("Крестики выиграли!")
            plot_board(env, None, showtext=False, fontq=fontq, fontx=fontx)
        if reward == -1:
            print("Нолики выиграли!")
            plot_board(env, None, showtext=False, fontq=fontq, fontx=fontx)

In [ ]:
env = TicTacToe(n_rows=3, n_cols=3, n_win=3)
plot_test_game(env, None, None, random_crosses=True, random_naughts=True, verbose=True, fontx=20)

Отлично. Игровая среда работает, давайте проведём случайную игру, посмотрим, как записываются состояния, вознаграждения и т.д.

In [ ]:
env = TicTacToe(n_rows=3, n_cols=3, n_win=3)
for i in range(9):
    print(env.step(env.random_available_action()))
print(env.step((0, 0)))

Реализуйте обычное (табличное) Q-обучение. Обучите стратегии крестиков и ноликов для доски 3х3.  

!["Q-learning")](https://hsto.org/webt/wf/6x/fi/wf6xfiyazgu0echvfsw8d9-oly4.png "Q-learning")

In [ ]:
def eps_greedy_strategy(env, epsilon, Q):
    """Эпсилон-жадная стратегия.
    С вероятностью эпсилон выбирает случайное действие, иначе - наиболее выгодное действие.
    
    Аргументы:
    env -- Окружение.
    epsilon -- Вероятность случайного действия.
    Q -- Выгода от действий в каждом состоянии dict: key - state, value - list values of actions.
    """
    state = env.getState()
    state = (state[0], state[2])
    if random.random() > epsilon:
        available = [(i, v) for i, v in enumerate(Q[state]) if env.is_available_action_int(i)]
        # print(available)
        best_available = list(filter(lambda x: x[1] == max(available, key=lambda x: x[1])[1], available))
        best_action = list(env.action_from_int(random.choice(best_available)[0]))
        # print("best action", best_action)
        return best_action
    else:
        # print("random action")
        return env.random_available_action()

In [ ]:
random_policy = lambda env: eps_greedy_strategy(env, 1.0, {})
best_policy = lambda env, Q: eps_greedy_strategy(env, 0.0, Q)

In [ ]:
def eps_generator(eps_start=1.0, eps_min=0.01, decay=0.99999):
    """Генератор элементов убывающей геометрической прогрессии с ограничением значения минимального элемента.
    
    Аргументы:
    eps_start -- Первый элемент последовательности.
    eps_min -- Минимальное значение элементов.
    decay -- Знаменатель прогрессии.
    """
    epsilon = eps_start
    yield epsilon
    command = None
    while True:
        if command == "restart":  # Начать сначала
            epsilon = eps_start
            yield epsilon
        else:
            epsilon = max(epsilon*decay, eps_min)  # Новый член последовательности
        command = (yield epsilon)

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
def play_episode(env, my_policy, opponent_policy, verbose=False):
    env.reset()
    if verbose:
        env.printBoard()
    done = False
    reward = 0
    
    while not done:
        action = my_policy(env)
        state, reward, done, _ = env.step(action)
        if verbose:
            env.printBoard()
        if done:
            break
        action = opponent_policy(env)
        state, reward, done, _ = env.step(action)
        if verbose:
            env.printBoard()
    return reward

In [ ]:
def policy_quality(env, my_policy, opponent_policy=random_policy, n_episodes=10000):
    reward = 0
    for i in range(n_episodes):
        reward += play_episode(env, my_policy, opponent_policy)
    return reward / n_episodes

Перед тем, как взяться за обучение, нужно подумать, против кого мы будем учиться играть, как будем оценивать качество, насколько жадным нужно быть при выборе действий и т.д.?  

Против кого играем?  
Можно было бы играть  
* против случайной стратегии;  
* против стратегии, которая параллельно обучается играть за другого игрока;  
* против какой-то фиксированной стратегии (эвристики, шаблоны и т.д.)  
Реализация третьего варианта нетривиальна, а первые два кажутся неплохими и простыми.  

Как оценивать качество игры?  
Очевидно, что для оценки качества надо сыграть много партий. Но с кем? Хотелось бы играть с таким игроком, который будет выискивать слабые места в стратегии противника, создавать неожиданные ситуации. Но реализовать такое - непростая задача. Если играть с какой-то фиксированной жёсткой стратегией, то кажется, что мы очень скудную оценку дадим, потому что оценим качество игры только в маленьком наборе ситуаций, которые эта стратегия умеет создавать, а это не совсем то, что хочется. Хочется, чтобы обученная стратегия умела действовать в большом количестве возможных ситуаций. Поэтому я решил, что качество буду оценивать, играя против случайной стратегии.  
Нужно заметить, что если я учусь играть с одной стратегией, а качество оцениваю, играя с другой стратегией, то игра будет не самая лучшая, а оценка заниженная, но я думаю, это не страшно, так даже интереснее.  
Так же нужно понимать, что при увеличении размеров доски и длины выстраиваемой линии случайная стратегия будет играть всё хуже и хуже.  

Насколько много надо исследовать?  
Мне кажется, в крестиках ноликах нужно исследовать много. Нужно за время обучения попытаться посмотреть, что будет в самых разных ситуациях. Если мы будем играть очень жадно против таких же жадных игроков, то мы изучим очень мало игровых состояний.  

В этом задании я решил параллельно учиться играть за обоих игроков и оценивать качество игры, играя со случайной стартегией.

In [ ]:
def qlearning(env, gamma=0.8, learning_rate=0.001, n_episodes=100000):
    Q = defaultdict(lambda: np.zeros(env.getTotalNumActions()))
    
    quality = []
    my_policy = lambda env: eps_greedy_strategy(env, 0, Q)
    quality.append(policy_quality(env, my_policy, n_episodes=n_episodes))
    
    eps_gen = eps_generator(eps_start=1.0, eps_min=0.01, decay=0.01**(1/n_episodes))
    
    pbar = tq.tqdm(total=n_episodes, desc='Episode', file=sys.stdout)
    for i in range(n_episodes):
        eps = next(eps_gen)
        state = env.reset()
        state = (state[0], state[2])
        
        reward_coef = 1.0
        if np.random.sample() > 0.5:
            # Other player
            action_other = eps_greedy_strategy(env, eps, Q)
            action_no_other = env.int_from_action(action_other)
            next_state_other, reward_other, done_other, _ = env.step(action_other)
            assert reward_other != -10
            state = (next_state_other[0], next_state_other[2])
            reward_coef = -reward_coef
        
        done = False
        reward = 0
        
        # While episode is not over
        while not done:
            # Choose action
            action = eps_greedy_strategy(env, eps, Q)
            action_no = env.int_from_action(action)
            # Do the action
            next_state, reward, done, _ = env.step(action)
            assert reward != -10
            reward *= reward_coef
            assert reward >= 0
            next_state = (next_state[0], next_state[2])
            if not done:
                # Other player
                action_other = eps_greedy_strategy(env, eps, Q)
                action_no_other = env.int_from_action(action_other)
                next_state_other, reward_other, done_other, _ = env.step(action_other)
                assert reward_other != -10
                reward = reward_other
                reward *= reward_coef
                assert reward <= 0
                next_state = (next_state_other[0], next_state_other[2])
                done = done_other
            # Update q_values
            td_target = reward + gamma * np.max(Q[next_state])
            td_error = td_target - Q[state][action_no]
            Q[state][action_no] += learning_rate * td_error
            # Update state
            state = next_state
        pbar.write(f"Episode {i} finish with reward {reward}.")
        if (i + 1) % (n_episodes // 10) == 0:
            my_policy = lambda env: eps_greedy_strategy(env, 0, Q)
            quality.append(policy_quality(env, my_policy, n_episodes=n_episodes))
            pbar.write(f"Policy reward vs random policy: {quality[-1]}. Current epsilon: {eps}. ")
        pbar.update(1)
        sleep(0.001)

    plt.plot(list(range(0, n_episodes + 1, n_episodes // 10)), quality)
    plt.ylabel("Reward vs random policy")
    plt.xlabel("Iteration of learning")
    
    best_policy = lambda env, Q: eps_greedy_strategy(env, 0, Q)
    return best_policy, Q

In [ ]:
env = TicTacToe(n_rows=3, n_cols=3, n_win=3)
pi, Q = qlearning(env, n_episodes=5000)

График выше мне кажется очень неплохим.  
Оценка качества производилось в игре за крестики. У крестиков на маленьких досках есть явное преимущество, которое даёт первый ход, поэтому уже в начале обучения мы чаще выигрывали случайную стратегию, чем проигрывали ей.  
Но уже после 3000 итераций мы научились играть довольно неплохо.  
Так же обратите внимание, что я к концу обучения всё таки сделал стратегию более жадной, это позволило увидеть, что обе стороны научились играть одинаково хорошо и почти всегда в конце обучения играли в ничью.  
Думаю, что подобрав правило изменения epsilon, мы бы смогли быстрее обучиться до приемлемого качества, наверное, за 2000 итераций можно было бы закончить.  

Попробуйте обучить стратегии крестиков и ноликов для доски 4х4 и/или 5х5.  
Disclaimer: начиная с пункта 2, задания для досок размера больше 4х4 могут потребовать большого терпения или более сложных вычислительных реализаций (например, параллелизации). Не хочу в этом задании непременно требовать ни того ни другого, так что если не будет получаться доучить до победных стратегий, не страшно -- но покажите, что со временем что-то всё-таки улучшается.

In [ ]:
env = TicTacToe(n_rows=4, n_cols=4, n_win=4)
pi, Q = qlearning(env, n_episodes=500000)

Результаты аналогичны тем, что были выше для доски 3х3, но потребовалось больше итераций обучения, а право первого хода даёт уже не такое большое преимущество, как раньше.

## Часть вторая: добавим нейронных сетей
Реализуйте нейронную сеть для метода DQN на доске для крестиков-ноликов. Не буду ограничивать фантазию, но кажется, что свёртки 3х3 здесь должны неплохо работать (в том числе обобщаться на доски размера побольше).  
Реализуйте DQN с нейронной сетью, обучите стратегии крестиков и ноликов. Замечание: скорее всего, experience replay потребуется сразу же.  

https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html  
https://mahowald.github.io/pytorch-dqn/  
https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/  
https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d  
https://neurohive.io/ru/tutorial/cnn-na-pytorch/  
https://ai.stackexchange.com/questions/6669/why-does-self-playing-tictactoe-not-become-perfect  


In [ ]:
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def store(self, exptuple):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = exptuple
        self.position = (self.position + 1) % self.capacity
       
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

В этом задании у нас есть больше свободы: больше параметров модели и процесса обучения, которые нужно подбирать или рассчитывать. Я плотно не занимался подбором параметров, выставлял их на глаз и если что-то не нравилось, корректировал тоже на глаз. Интуитивно действовал. Вообще, относительно того какую сделать модель, как её учить вопросов больше, чем ответов. Поэтому уверен, что при более грамотном подходе можно значительно улучшить количественные показатели, но качественно процесс обучения получилось продемонстрировать.  

По рекомендации из условия задачи решил добавить свёрточный слой, а после него поместил два полносвязных слоя.  
Я думаю, размеры слоёв должны как-то зависить от размера доски, поэтому попробовал их связать. 

In [ ]:
class Network(nn.Module):
    def __init__(self, field_size=(3, 3), conv_size=3):
        nn.Module.__init__(self)
        conv_number = int((field_size[0] + field_size[1] + 2) * 1.5)
        hidden_layer_size = conv_number * field_size[0] * field_size[1] * 2
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, conv_number, kernel_size=conv_size, stride=1, padding=0),
            nn.Sigmoid()
        )
        self.fc1 = nn.Sequential(
            nn.Linear((1 + field_size[0] - conv_size) * (1 + field_size[1] - conv_size) * conv_number, hidden_layer_size),
            nn.Sigmoid()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(hidden_layer_size, field_size[0] * field_size[1]),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.conv1(x)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

Решил независимо учить две модели за крестики и за нолики. Их можно было бы объединить и аккуратно обучить одну модель, но мне понравилось так.  
Я решил сделать побольше исследования новых состояний, поэтому сделал стратегии очень нежадными. Но при этом мне показалось естестенным сохранить хотя бы небольшую динамику изменения epsilon, поэтому я уменьшаю epsilon с 0.95 до 0.85 в процессе обучения. Случайности много, но при этом хорошие ходы будут фигурировать в обучении чаще, чем было бы при использовании абсолютно случайных стратегий.  

In [ ]:
class TicTacToeDQN():
    def __init__(self, n_rows=3, n_cols=3, n_win=3):
        self.env = TicTacToe(n_rows, n_cols, n_win)
        self.model_x = Network((n_rows, n_cols))
        self.model_o = Network((n_rows, n_cols))
        self.memory_x = ReplayMemory(1000 * n_rows * n_cols)
        self.memory_o = ReplayMemory(1000 * n_rows * n_cols)
        self.optimizer_x = optim.Adam(self.model_x.parameters(), 0.001)
        self.optimizer_o = optim.Adam(self.model_o.parameters(), 0.001)
        
        self.gamma = 0.95
        self.batch_size = 64
        
        self.eps_init, self.eps_final, self.eps_decay = 0.95, 0.85, 200
        self.eps_threshold = self.eps_init
        self.eps_gen = None
        self.num_step = 0

    def select_greedy_action(self, model, state):
        with torch.no_grad():
            values = model(state.expand(1, 1, self.env.n_rows, self.env.n_cols))[0].tolist()
            available = [(i, v) for i, v in enumerate(values) if self.env.is_available_action_int(i)]
            best_available = list(filter(lambda x: x[1] == max(available, key=lambda x: x[1])[1], available))
            best_action = random.choice(best_available)[0]
            return torch.tensor([[best_action]])
    
    def select_random_action(self):
        return torch.tensor([[self.env.random_available_action_no()]], dtype=torch.int64)

    def select_action(self, model, state):
        with torch.no_grad():
            sample = random.random()
            self.num_step += 1
            if sample > self.eps_threshold:
                return self.select_greedy_action(model, state)
            else:
                return self.select_random_action()
    
    def state_to_tensor(self, env_state):
        state = list(map(int, env_state[0]))
        state_tensor = torch.tensor([state], dtype=torch.float32).reshape(self.env.n_rows, self.env.n_cols)
        return state_tensor
    
    def do_step(self, current_state_tensor, greedy=False, random_step=False, render=False):
        if self.env.curTurn == 1:
            model = self.model_x
            if render: print("Ходят крестики:")
        else:
            model = self.model_o
            if render: print("Ходят нолики:")
        if random_step:
            if render: print("Случайный ход")
            action = self.select_random_action()
        else:
            if greedy:
                if render: print("Жадная стратегия")
                action = self.select_greedy_action(model, current_state_tensor)
            else:
                if render: print("Почти жадная стратегия")
                action = self.select_action(model, current_state_tensor)
        new_state, reward, done, _ = self.env.step(self.env.action_from_int(action.numpy()[0][0]))
        if render:
            self.env.printBoard()
        return action, new_state, reward, done
        
    
    def run_episode(self, e=0, do_learning=True, greedy=False, render=False, vs_random=False, verbose=True, side="random"):
        self.model_x.train(False)
        self.model_o.train(False)
        
        state = self.env.reset()
        state_tensor = self.state_to_tensor(state)
        if render:
            self.env.printBoard()
        
        if do_learning:
            if self.eps_gen is None:
                self.eps_threshold = self.eps_final + (self.eps_init - self.eps_final) * math.exp(-1. * self.num_step / self.eps_decay)
            else:
                self.eps_threshold = next(self.eps_gen)
        
        reward_coef = 1.0
        if side == "o" or (side == "random" and random.random() > 0.5):
            if render:
                print("Играем за нолики")
            _, state, _, _ = self.do_step(state_tensor, greedy=True, random_step=vs_random, render=render)
            state_tensor = self.state_to_tensor(state)
            reward_coef = -reward_coef
        
        while True:
            action, next_state, reward, done = self.do_step(state_tensor, greedy=greedy, random_step=False, render=render)
            next_state_tensor = self.state_to_tensor(next_state)
            assert reward != -10
            reward *= reward_coef
            assert reward >= 0
                
            if not done:
                # Other player
                action_other, next_state, reward, done = self.do_step(next_state_tensor, greedy=True, random_step=vs_random, render=render)
                next_state_tensor = self.state_to_tensor(next_state)
                assert reward != -10
                reward *= reward_coef
                assert reward <= 0

            if do_learning:
                transition = (state_tensor, action, next_state_tensor, torch.tensor([reward], dtype=torch.float32))
                if reward_coef > 0:
                    self.memory_x.store(transition)
                else:
                    self.memory_o.store(transition)
            state = next_state
            state_tensor = self.state_to_tensor(state)
            
            if do_learning:
                self.learn()
            
            if done:
                if verbose: print("\tEpisode %d finished with reward %d. Current epsilon == %f." % (e, reward, self.eps_threshold))
                break
            
        return reward
                
    def run(self, n_episodes=10000, render=False, vs_random=False):
        if self.eps_gen is None:
            self.eps_gen = eps_generator(eps_start=self.eps_init, eps_min=self.eps_final, decay=(self.eps_final / self.eps_init)**(1/n_episodes))
        
        print("%s\tStarting training for %d episodes..." % (datetime.now().time(), n_episodes))
        quality = []
        quality.append(self.mean_reward_vs_random_policy(n_episodes))
        print(f"Current quality == {quality[-1]}")
        for e in range(n_episodes):
            self.run_episode(e, render=render, vs_random=vs_random)
            if (e + 1) % (n_episodes // 10) == 0:
                quality.append(self.mean_reward_vs_random_policy(n_episodes))
                print(f"Current quality == {quality[-1]}")
        print("%s\t\t...done!" % (datetime.now().time()))
        
        plt.plot(list(range(0, n_episodes + 1, n_episodes // 10)), quality)
        plt.ylabel("Reward vs random policy")
        plt.xlabel("Iteration of learning")
        
        return quality

    def learn(self):
        if len(self.memory_x) < self.batch_size or len(self.memory_o) < self.batch_size:
            return

        self.model_x.train(True)
        self.model_o.train(True)
        
        x = (self.model_x, self.memory_x, self.optimizer_x)
        o = (self.model_o, self.memory_o, self.optimizer_o)
        
        for model, memory, optimizer in [x, o]:
            # берём мини-батч из памяти
            transitions = memory.sample(self.batch_size)
            batch_state, batch_action, batch_next_state, batch_reward = zip(*transitions)
            batch_state = Variable(torch.cat([state.expand(1, 1, self.env.n_rows, self.env.n_cols) for state in batch_state]))
            batch_action = Variable(torch.cat(batch_action))
            batch_reward = Variable(torch.cat(batch_reward))
            batch_next_state = Variable(torch.cat([state.expand(1, 1, self.env.n_rows, self.env.n_cols) for state in batch_next_state]))

            # считаем значения функции Q
            Q = model(batch_state).gather(1, batch_action).reshape([self.batch_size])

            # оцениваем ожидаемые значения после этого действия
            Qmax = model(batch_next_state).detach().max(1)[0]
            Qnext = batch_reward + (self.gamma * Qmax)

            # и хотим, чтобы Q было похоже на Qnext -- это и есть суть Q-обучения
            loss = F.smooth_l1_loss(Q, Qnext)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    def mean_reward_vs_random_policy(self, n_episodes=1000, side="x"):
        self.model_x.train(False)
        self.model_o.train(False)
        rewards = 0
        for i in range(n_episodes):
            rewards += self.run_episode(e=0, do_learning=False, greedy=True, render=False, vs_random=True, verbose=False, side=side)
        return rewards / n_episodes

In [ ]:
%%time
torch.manual_seed(1)
dqn = TicTacToeDQN(3, 3, 3)
quality = dqn.run(10000, vs_random=False)

Ну... График не очень красивый, но видно, что результат от обучения есть: мы играем значительно лучше, чем играет случайная стратегия.  
Смотреть на вознаграждения в процессе обучения здесь не имеет большого смысла, потому что стратегии совершают очень много случайных действий.

Для доски побольше:

In [ ]:
%%time
torch.manual_seed(1)
dqn = TicTacToeDQN(4, 4, 4)
quality = dqn.run(10000, vs_random=False)

Опять не очень красиво, и, похоже, можно было продолжать ещё учить, но эффект от обучения виден и на этом этапе.

Реализуйте Double DQN и/или Dueling DQN.

https://towardsdatascience.com/double-deep-q-networks-905dd8325412  
https://theaisummer.com/Taking_Deep_Q_Networks_a_step_further/  
https://www.freecodecamp.org/news/improvements-in-deep-q-learning-dueling-double-dqn-prioritized-experience-replay-and-fixed-58b130cc5682/  
https://en.wikipedia.org/wiki/Q-learning#Double_Q-learning  
https://medium.com/@leosimmons/double-dqn-implementation-to-solve-openai-gyms-cartpole-v-0-df554cd0614d  


Здесь ничего не придумывал, скопировал код для DQN и изменил функцию обучения. Реализован один из самых простых вариантов DoubleDQN (кажется, вариант реализации, представленный в первой публикации о DoubleDQN): веса второй модели подтягивают к первой жёстко, но не на каждой итерации. Позже были предложены более красивые реализации с мягким подтягиванием весов одной модели к другой, но я решил попробовать простой вариант, и он неплохо сработал, я его таким и оставил.

In [ ]:
class TicTacToeDoubleDQN():
    def __init__(self, n_rows=3, n_cols=3, n_win=3):
        self.env = TicTacToe(n_rows, n_cols, n_win)
        self.models_x = [Network((n_rows, n_cols)), Network((n_rows, n_cols))]
        self.models_o = [Network((n_rows, n_cols)), Network((n_rows, n_cols))]
        self.memory_x = ReplayMemory(1000 * n_rows * n_cols)
        self.memory_o = ReplayMemory(1000 * n_rows * n_cols)
        learning_rate = 0.001
        self.optimizers_x = [optim.Adam(self.models_x[0].parameters(), learning_rate), 
                             optim.Adam(self.models_x[1].parameters(), learning_rate)]
        self.optimizers_o = [optim.Adam(self.models_o[0].parameters(), learning_rate), 
                             optim.Adam(self.models_o[1].parameters(), learning_rate)]
        
        self.gamma = 0.95
        self.batch_size = 64
        
        self.eps_init, self.eps_final, self.eps_decay = 0.95, 0.85, 200
        self.eps_threshold = self.eps_init
        self.eps_gen = None
        self.num_step = 0
        self.update_count = 0

    def select_greedy_action(self, model, state):
        with torch.no_grad():
            values = model(state.expand(1, 1, self.env.n_rows, self.env.n_cols))[0].tolist()
            available = [(i, v) for i, v in enumerate(values) if self.env.is_available_action_int(i)]
            best_available = list(filter(lambda x: x[1] == max(available, key=lambda x: x[1])[1], available))
            best_action = random.choice(best_available)[0]
            return torch.tensor([[best_action]])
    
    def select_random_action(self):
        return torch.tensor([[self.env.random_available_action_no()]], dtype=torch.int64)

    def select_action(self, model, state):
        with torch.no_grad():
            sample = random.random()
            self.num_step += 1
            if sample > self.eps_threshold:
                return self.select_greedy_action(model, state)
            else:
                return self.select_random_action()
    
    def state_to_tensor(self, env_state):
        state = list(map(int, env_state[0]))
        state_tensor = torch.tensor([state], dtype=torch.float32).reshape(self.env.n_rows, self.env.n_cols)
        # state_tensor = (state_tensor + 1.0) * self.env.curTurn
        return state_tensor
    
    def do_step(self, current_state_tensor, greedy=False, random_step=False, render=False):
        if self.env.curTurn == 1:
            model = self.models_x[0]
            if render: print("Ходят крестики:")
        else:
            model = self.models_o[0]
            if render: print("Ходят нолики:")
        if random_step:
            if render: print("Случайный ход")
            action = self.select_random_action()
        else:
            if greedy:
                if render: print("Жадная стратегия")
                action = self.select_greedy_action(model, current_state_tensor)
            else:
                if render: print("Почти жадная стратегия")
                action = self.select_action(model, current_state_tensor)
        new_state, reward, done, _ = self.env.step(self.env.action_from_int(action.numpy()[0][0]))
        if render:
            self.env.printBoard()
        return action, new_state, reward, done
        
    
    def run_episode(self, e=0, do_learning=True, greedy=False, render=False, vs_random=False, verbose=True, side="random"):
        self.models_x[0].train(False)
        self.models_x[1].train(False)
        self.models_o[0].train(False)
        self.models_o[1].train(False)
        
        state = self.env.reset()
        state_tensor = self.state_to_tensor(state)
        if render:
            self.env.printBoard()
        
        if do_learning:
            if self.eps_gen is None:
                self.eps_threshold = self.eps_final + (self.eps_init - self.eps_final) * math.exp(-1. * self.num_step / self.eps_decay)
            else:
                self.eps_threshold = next(self.eps_gen)
        
        reward_coef = 1.0
        if side == "o" or (side == "random" and random.random() > 0.5):
            if render:
                print("Играем за нолики")
            _, state, _, _ = self.do_step(state_tensor, greedy=True, random_step=vs_random, render=render)
            state_tensor = self.state_to_tensor(state)
            reward_coef = -reward_coef
        
        while True:
            action, next_state, reward, done = self.do_step(state_tensor, greedy=greedy, random_step=False, render=render)
            next_state_tensor = self.state_to_tensor(next_state)
            assert reward != -10
            reward *= reward_coef
            assert reward >= 0
                
            if not done:
                # Other player
                action_other, next_state, reward, done = self.do_step(next_state_tensor, greedy=True, random_step=vs_random, render=render)
                next_state_tensor = self.state_to_tensor(next_state)
                assert reward != -10
                reward *= reward_coef
                assert reward <= 0

            if do_learning:
                transition = (state_tensor, action, next_state_tensor, torch.tensor([reward], dtype=torch.float32))
                if reward_coef > 0:
                    self.memory_x.store(transition)
                else:
                    self.memory_o.store(transition)
            state = next_state
            state_tensor = self.state_to_tensor(state)
            
            if do_learning:
                self.learn()
            
            if done:
                if verbose: print("\tEpisode %d finished with reward %d. Current epsilon == %f." % (e, reward, self.eps_threshold))
                break
            
        return reward
                
    def run(self, n_episodes=10000, render=False, vs_random=False):
        if self.eps_gen is None:
            self.eps_gen = eps_generator(eps_start=self.eps_init, eps_min=self.eps_final, decay=(self.eps_final / self.eps_init)**(1/n_episodes))
        
        print("%s\tStarting training for %d episodes..." % (datetime.now().time(), n_episodes))
        quality = []
        quality.append(self.mean_reward_vs_random_policy(n_episodes))
        print(f"Current quality == {quality[-1]}")
        for e in range(n_episodes):
            self.run_episode(e, render=render, vs_random=vs_random)
            if (e + 1) % (n_episodes // 10) == 0:
                quality.append(self.mean_reward_vs_random_policy(n_episodes))
                print(f"Current quality == {quality[-1]}")
        print("%s\t\t...done!" % (datetime.now().time()))
        
        plt.plot(list(range(0, n_episodes + 1, n_episodes // 10)), quality)
        plt.ylabel("Reward vs random policy")
        plt.xlabel("Iteration of learning")
        
        return quality

    def update_target_network(self):
        self.update_count += 1
        if self.update_count % 5 == 0:
            self.models_x[1].load_state_dict(self.models_x[0].state_dict())
            self.models_x[1].load_state_dict(self.models_x[0].state_dict())
    
    def learn(self):
        if len(self.memory_x) < self.batch_size or len(self.memory_o) < self.batch_size:
            return

        self.models_x[0].train(True)
        self.models_x[1].train(True)
        self.models_o[0].train(True)
        self.models_o[1].train(True)
        
        x = (self.models_x, self.memory_x, self.optimizers_x)
        o = (self.models_o, self.memory_o, self.optimizers_o)
        
        for models, memory, optimizers in [x, o]:
            # берём мини-батч из памяти
            transitions = memory.sample(self.batch_size)
            batch_state, batch_action, batch_next_state, batch_reward = zip(*transitions)
            batch_state = Variable(torch.cat([state.expand(1, 1, self.env.n_rows, self.env.n_cols) for state in batch_state]))
            batch_action = Variable(torch.cat(batch_action))
            batch_reward = Variable(torch.cat(batch_reward))
            batch_next_state = Variable(torch.cat([state.expand(1, 1, self.env.n_rows, self.env.n_cols) for state in batch_next_state]))

            # считаем значения функции Q
            Q = models[0](batch_state).gather(1, batch_action).reshape([self.batch_size])

            # оцениваем ожидаемые значения после этого действия
            Qmax = models[0](batch_next_state).detach()[:, torch.argmax(models[1](batch_next_state).detach(), 1)][: ,0]
            Qnext = batch_reward + (self.gamma * Qmax)
            
            loss = F.smooth_l1_loss(Q, Qnext)
            optimizers[0].zero_grad()
            loss.backward()
            optimizers[0].step()
            
            self.update_target_network()

    def mean_reward_vs_random_policy(self, n_episodes=1000, side="x"):
        self.models_x[0].train(False)
        self.models_x[1].train(False)
        self.models_o[0].train(False)
        self.models_o[1].train(False)
        rewards = 0
        for i in range(n_episodes):
            rewards += self.run_episode(e=0, do_learning=False, greedy=True, render=False, vs_random=True, verbose=False, side=side)
        return rewards / n_episodes

In [ ]:
%%time
torch.manual_seed(1)
random.seed(1)
ddqn3 = TicTacToeDoubleDQN(3, 3, 3)
quality = ddqn3.run(10000)
print(quality)

Эта кривая значительно красивее, чем та, что была для DQN. Видно, что процесс обучения стал стабильнее, а то же качество может быть достигнуто за меньшее число итераций.

In [ ]:
%%time
torch.manual_seed(1)
random.seed(1)
ddqn4 = TicTacToeDoubleDQN(4, 4, 4)
quality = ddqn4.run(10000)
print(quality)

Аналогично для доски 4х4: стало хорошо. Такой график уже радует. Обучается быстрее и стабильнее.

## Часть третья: расширим и углубим поиск
Крестики-нолики -- это, конечно, далеко не го, и обычный альфа-бета поиск с отсечением здесь наверняка может работать идеально вплоть до довольно больших досок. Однако мы всё-таки для этого учебного задания будем реализовывать более практически релевантный метод MCTS -- заодно фактически получится и упражнение на многоруких бандитов.  
Реализуйте rollouts со случайной стратегией и (опционально) rollouts с неслучайной, но простой стратегией (например, основанной на дополнении нескольких паттернов или на Q-функции, которая у вас получилась в первом пункте).  

https://robotics.stackexchange.com/questions/16596/what-is-the-definition-of-rollout-in-neural-network-or-openai-gym  
https://arxiv.org/pdf/1910.00120.pdf  
https://towardsdatascience.com/monte-carlo-tree-search-in-reinforcement-learning-b97d3e743d0f  
https://towardsdatascience.com/monte-carlo-tree-search-implementing-reinforcement-learning-in-real-time-game-player-25b6f6ac3b43  

Напишем функцию, которая проигрывает партию с текущего состояния до конца в соответствии с заданными стратегиями игры:

In [ ]:
def rollout(env, x_policy, o_policy, my_side='x', smart_reward=False, render=False):
    policies = [None, x_policy, o_policy]  # для удобства индексации с помощью curTurn
    reward = 0
    done = env.gameOver
    if render: env.printBoard()
    while not done:
        _, reward, done, _ = env.step(policies[env.curTurn](env))
        if render: env.printBoard()
        assert reward != -10
    if smart_reward and my_side == 'o':
        reward = -reward
    return reward

def random_policy(env):
    return env.random_available_action()

Сыграем несколько партий:

In [ ]:
env = TicTacToe(3, 3, 3)

env.reset()
print(rollout(env, random_policy, random_policy))

env.reset()
print(rollout(env, random_policy, random_policy))

env.reset()
print(rollout(env, random_policy, random_policy))

Давайте теперь попробуем играть, проигрывая в уме продолжение партии с помощью ранее написанной функции, и таким образом выбирая наилучшие действия.

Для начала без проигрывания партий в уме (играет случайная стратегия, не имеющая представления, как лучше ходить, против обученной DoubleDQN):

In [ ]:
def ddqn3_policy_x(env):
    ddqn3.env = env
    return env.action_from_int(ddqn3.select_greedy_action(ddqn3.models_x[0], ddqn3.state_to_tensor(env.getState())).numpy()[0][0])
    
def mean_reward_without_rollout(n_episodes=100):
    env = TicTacToe(3, 3, 3)
    cum_rewards = 0
    for _ in range(n_episodes):
        env.reset()
        cum_rewards += rollout(env, ddqn3_policy_x, random_policy)
    return cum_rewards / n_episodes

print(mean_reward_without_rollout())

Видим что обученная ранее стратегия играет лучше нашей случайной.  
А теперь сделаем следующее: у нас нет информации о том, какой ход лучше, но есть информация о том, как противник отвечает на наши ходы, т.е. мы имеем возможность проиграть партию в уме для всех доступных действий. Сделаем это.  
Сейчас наша стратегия станет жёсткой: мы будем однозначно и закономерно выбирать ходы. И стратегия противника жёсткая. Поэтому играть много партий не нужно - они будут одинаковые. Сыграем одну: 

In [ ]:
def rollout_policy(env):
    available_actions = env.getEmptySpaces()
    best_action = None
    best_reward = -10
    for action in available_actions:
        env_copy = copy.deepcopy(env)
        env_copy.step(action)
        rollout_reward = rollout(env_copy, ddqn3_policy_x, rollout_policy, 'o', True)
        if rollout_reward > best_reward:
            best_reward = rollout_reward
            best_action = action
    return best_action

def mean_reward_with_rollout(n_episodes=100, render=False):
    env = TicTacToe(3, 3, 3)
    cum_rewards = 0
    for _ in tq.tqdm(range(n_episodes)):
        env.reset()
        env.step(random_policy(env))
        cum_rewards += rollout(env, ddqn3_policy_x, rollout_policy, render=render)
    return cum_rewards / n_episodes

print(mean_reward_with_rollout(1, render=False))

Видим, что имея возможность проигрывать продолжение партий в уме, нолики выиграли.

Реализуйте MCTS-поиск с этими rollouts для крестиков-ноликов на досках разного размера, сравните полученные стратегии между собой и со стратегиями, обученными в первых двух частях.

In [ ]:
def replace(s, position, character):
    return s[:position] + character + s[position+1:]

In [ ]:
C = 0.8

class Node():
    def __init__(self):
        self.parent = None
        self.children = {}
        self.visits = 0
        self.value = 0
        self.state = None
        self.index = 0
        self.storage = None
    
    def ucb(self):
        if self.visits == 0:
            return float('inf')
        return self.value / self.visits + C * math.sqrt(math.log(self.storage[self.parent].visits) / self.visits)
    
    def create_children(self):
        if self.is_leaf():
            np_state = np.array(list(map(int, self.state)))
            available_actions = np.where(np_state == 1)[0]
            if (len(self.state) - len(available_actions)) % 2 == 0:
                curTurn = 1
            else:
                curTurn = -1
            for action in available_actions:
                child = Node()
                child.parent = self.index
                child.state = replace(self.state, action, str(curTurn + 1))
                child.index = len(self.storage)
                child.storage = self.storage
                self.children[action] = child.index
                self.storage.append(child)
    
    def is_leaf(self):
        return len(self.children) == 0
    
    def select_child(self, how="best"):
        eps = 0.7
        if how == "best":
            fun = max
            characteristic = lambda x: self.storage[x[1]].ucb()
        elif how == "worse":
            fun = min
            characteristic = lambda x: self.storage[x[1]].ucb()
        elif how == "random":
            return random.choice(list(self.children.items()))
        elif how == "eps_greedy_best":
            if random.random() < eps:
                return random.choice(list(self.children.items()))
            else:
                fun = max
                characteristic = lambda x: self.storage[x[1]].ucb()
        elif how == "eps_greedy_worse":
            if random.random() < eps:
                return random.choice(list(self.children.items()))
            else:
                fun = min
                characteristic = lambda x: self.storage[x[1]].ucb()  
        else:
            fun = None
            characteristic = None
        if not self.is_leaf():
            return max(self.children.items(), key=characteristic)

In [ ]:
class Mcts():
    def __init__(self, n_rows=3, n_cols=3, n_win=3):
        self.env = TicTacToe(n_rows, n_cols, n_win)
        self.env.reset()

        self.nodes = []

        root = Node()
        root.state = self.env.getState()[0]
        root.index = 0
        root.storage = self.nodes
        self.nodes.append(root)
        
        self.current_node = 0
        
    def next_step(self):
        hows = [None, "best", "worse"]
        action, self.current_node = self.nodes[self.current_node].select_child(hows[self.env.curTurn])
        self.env.step(self.env.action_from_int(action))
    
    def selection(self):
        while not self.nodes[self.current_node].is_leaf():
            self.next_step()
    
    def expansion(self):
        if not self.env.gameOver:
            if self.nodes[self.current_node].visits > 0:
                self.nodes[self.current_node].create_children()
                self.next_step()
    
    def simulation(self):
        return rollout(copy.deepcopy(self.env), random_policy, random_policy)
    
    def backpropagation(self, r):
        while self.nodes[self.current_node].parent is not None:
            self.nodes[self.current_node].visits += 1
            self.nodes[self.current_node].value += r
            self.current_node = self.nodes[self.current_node].parent
        self.nodes[self.current_node].visits += 1
        self.nodes[self.current_node].value += r
    
    def learn(self, n_episodes=100000):
        for _ in tq.tqdm(range(n_episodes)):
            assert self.current_node == 0
            self.env.reset()
            self.selection()
            self.expansion()
            r = self.simulation()
            self.backpropagation(r)
            
    def play_greedy_game(self):
        self.env.reset()
        self.env.printBoard()
        self.current_node = 0
        hows = [None, "best", "worse"]
        while not self.nodes[self.current_node].is_leaf():
            action, self.current_node = self.nodes[self.current_node].select_child(hows[self.env.curTurn])
            self.env.step(self.env.action_from_int(action))
            self.env.printBoard()
    
    def mean_reward_vs_random_policy(self, n_episodes=1000, side='x'):
        if side == 'x':
            hows = [None, "best", "random"]
        elif side == 'o':
            hows = [None, "random", "worse"]
        else:
            hows = None
        cum_rewards = 0
        count_games = 0
        for _ in range(n_episodes):
            self.env.reset()
            self.current_node = 0
            while not self.nodes[self.current_node].is_leaf():
                action, self.current_node = self.nodes[self.current_node].select_child(hows[self.env.curTurn])
                _, reward, done, _ = self.env.step(self.env.action_from_int(action))
            if done:
                cum_rewards += reward
                count_games += 1
        mean_reward = cum_rewards / count_games
        print(f"Mean reward vs random policy equal {mean_reward} after {count_games} games.")
        return mean_reward

Реализовали построение дерева. Попробуем построить и посмотреть результаты:

In [ ]:
mcts3 = Mcts(3, 3, 3)
mcts3.learn()

Посмотрим на партию жадных стратегий, пользующихся построенным деревом:

In [ ]:
mcts3.play_greedy_game()

Мы строили дерево недолго, некоторые его листы могут не быть терминальными состояниями в игре. Т.е. может получиться так, что до листа дерева мы прошли,а игра ещё не закончилась. В оценке среднего выигрыша я такие партии игнорировал и брал для оценки только законченные партии. Оценим средний выигрыш против случайной стратегии:

In [ ]:
mcts3.mean_reward_vs_random_policy()

Аналогично для 4х4:

In [ ]:
mcts4 = Mcts(4, 4, 4)
mcts4.learn(1000000)

In [ ]:
mcts4.play_greedy_game()

In [ ]:
mcts4.mean_reward_vs_random_policy(1000)

Результаты 4х4 вызывают некоторые сомнения. Слишком мало игр доигрались до конца.  
Попробую найти этому объяснение. 
* Когда алгоритм выбирает какой-то путь в дереве, он запускает на этом пути rollout. Происходит игра двух случайных стратегий, чтобы определить, к какому результату может привести этот путь. Игра 4х4 достаточно длинная и сложная, чтобы случайная стратегия выигрывала в неё у другой случайной стратегии крайне редко. Поэтому все новые пути, которые даже могли быть хорошими, кажутся алгоритму примерно одинаково неважными. И лишь иногда так получается, что случайные симуляции выделяют какой-то путь на фоне остальных, сыграв несколько показательных партий по этому направлению. После этого алгоритм начинает сильнее углубляться по этим путям и уделять меньше внимания остальным путям в дереве. А т.к. игра 4х4 достаточно длинная (дерево глубокое), то на путях, которым уделяется недостаточно внимания конец игры достигнут будет очень нескоро.  
* Максимальное количество узлов в дереве меняется примерно как факториал количества клеток на доске. Это очень много, начиная с досок размеров 4х4.  

Получаем, что из 1000 партий завершились не многие. Но завершились победой )  
Что с этим можно сделать, как сделать результаты красивее?  
Кажется, что если строить дерево значительно дольше, то мы должны исследовать больше путей в дереве, но я пробовал учить несколько часов, это не сильно помогло, а учить ещё дольше в рамках домашней работы не очень хочется, хочется предложить качественное, а не количественное решение. Я пробовал менять условия выбора действия (делать их менее жадными), но всё это слабо меняет результат, потому что дерево очень большое. Можно попробовать выстраивать линию не из 4 символов, а из трёх, тогда игра будет заканчиваться раньше. Посмотрим...

In [ ]:
mcts4short = Mcts(4, 4, 3)
mcts4short.learn(1000000)

In [ ]:
mcts4short.play_greedy_game()

In [ ]:
mcts4short.mean_reward_vs_random_policy(1000)

Уже лучше. По такому количеству игр уже можно делать выводы. Думаю, пока я на этом остановлюсь.

Качество игры примерно такое же, как то, которое мы получали, учась другими методами, это похоже на правду.  

Что я понял об MCTS:  
```
+ Просто реализовать
+ Достаточно знать лишь правила игры
- Требователен к ресурсам
- Для сложных игр использовать mcts не просто
```